In [5]:
import google.generativeai as genai
import json
from datetime import datetime
from typing import Dict, List, Any
import os

class MOBYReportGenerator:
    """MOBY 주간 보고서 자동 생성기 (Gemini 2.5 Flash)"""
    
    def __init__(self, api_key: str = None):
        """
        Args:
            api_key: Gemini API 키 (없으면 환경변수 GEMINI_API_KEY 사용)
        """
        api_key = api_key or os.getenv('GEMINI_API_KEY')
        genai.configure(api_key=api_key)
        
        # Gemini 2.5 Flash 모델 설정
        self.model = genai.GenerativeModel(
            model_name='gemini-2.5-flash',
            generation_config={
                'temperature': 0.18,  # 일관성을 위해 낮은 temperature
                'top_p': 0.8,
                'top_k': 40,
                'max_output_tokens': 8192,
            }
        )
    
    def generate_report(self, report_data: Dict[str, Any]) -> str:
        """
        보고서 데이터를 기반으로 주간 보고서 생성
        
        Args:
            report_data: 보고서 생성에 필요한 모든 데이터
                - period: {"start": "2025-11-07 09:00", "end": "2025-11-14 09:00"}
                - equipment: "Conveyor A-01"
                - sensor_stats: 센서별 통계
                - alarms: 알람 목록
                - mlp_anomalies: MLP 탐지 결과
                - if_anomalies: Isolation Forest 탐지 결과
                - correlations: 센서 간 상관계수
                
        Returns:
            생성된 마크다운 보고서 문자열
        """
        prompt = self._build_prompt(report_data)
        
        response = self.model.generate_content(prompt)
        return response.text
    
    def _build_prompt(self, data: Dict[str, Any]) -> str:
        """보고서 생성을 위한 프롬프트 구성"""
        
        prompt = f"""당신은 산업 설비 모니터링 전문가입니다. 아래 센서 데이터와 이상 탐지 결과를 바탕으로, 
엔지니어링 팀을 위한 주간 보고서를 작성해주세요.

# 보고서 작성 지침

## 형식 요구사항
- 마크다운 형식으로 작성
- 제공된 템플릿 구조를 정확히 따를 것
- 표는 정렬된 형태로 작성

## 내용 요구사항
1. **Executive Summary**: 3-4문장으로 핵심 발견사항 요약
   - 주요 센서 상관관계
   - 이상 탐지 건수 및 심각도
   - 즉시 조치 필요 여부
   
2. **센서별 통계**: 제공된 수치를 표로 정리
   - 임계값 초과 시 **굵은 글씨**로 강조
   
3. **이상 탐지 상세**: 각 이상에 대해
   - 발생 시각 및 유형
   - 원시 데이터 JSON 포함
   - **물리적 해석**: 왜 이 패턴이 발생했는지, 설비 관점에서 설명
   - 구체적 권장 조치
   
4. **상관 분석 인사이트**: 
   - 센서 간 관계의 **공학적 의미** 해석
   - 시간적 패턴 분석
   - 근본 원인 추론
   
5. **권장 사항**: 우선순위별 분류 (High/Medium/Ongoing)

## 톤 및 스타일
- 전문적이고 명확한 기술 문서 스타일
- 불확실한 추측보다는 데이터 기반 분석
- 과장하지 않되, 위험 신호는 명확히 지적

---

# 입력 데이터 구조
아래 JSON은 다음 키를 포함합니다:
- `metadata`: 보고 기간, 설비명, 생성 시각
- `sensor_stats`: 센서별 통계 (온도, 습도, 진동, 가속도, 자이로, 음압, 기압)
- `alarms`: 규칙 기반 임계값 초과 알람 목록
- `mlp_anomalies`: MLP 모델이 탐지한 알려진 이상 패턴
- `if_anomalies`: Isolation Forest가 탐지한 미지의 이상 (Novelty)
- `correlations`: 센서 간 상관계수 및 해석

# 입력 데이터
```json
{json.dumps(data, indent=2, ensure_ascii=False)}
```

---

# 보고서 템플릿

아래 구조를 따라 보고서를 작성하세요:
```markdown
# 📘 MOBY 설비 모니터링 · 이상 탐지 주간 보고서

| 항목 | 내용 |
| :--- | :--- |
| **보고 기간** | [자동 입력] |
| **설비** | [자동 입력] |
| **생성 일시** | [자동 입력] |

---

## 1. 📊 Executive Summary

[핵심 발견사항을 3-4문장으로 요약]

---

## 2. 📈 센서별 통계 요약

### 2.1 온도/습도 (DHT11)
[표 작성]

### 2.2 진동 (Vibration)
[표 작성]

### 2.3 가속도/자이로 (MPU-6050)
[표 작성]

### 2.4 음압 (Sound)
[표 작성]

### 2.5 기압 (Pressure)
[표 작성]

---

## 3. ⚠️ 알람 및 이상 탐지 상세

### 3.1 규칙 기반 센서 알람
[표 작성]

### 3.2 MLP 기반 이상 탐지 (알려진 이상)
[각 이상별 상세 분석]

### 3.3 Isolation Forest 기반 미지의 이상 (Novelty)
[Novelty 패턴 분석]

---

## 4. 🔗 상관 분석 및 주간 인사이트

### 4.1 센서 간 상관계수
[표 작성]

### 4.2 주요 인사이트
[번호 목록으로 핵심 인사이트 정리]

---

## 5. 🛠️ 권장 사항

### 즉시 조치 (High Priority)
[체크박스 항목]

### 중기 조치 (Medium Priority)
[체크박스 항목]

### 모니터링 지속 (Ongoing)
[체크박스 항목]
```

이제 위 템플릿에 맞춰 보고서를 작성해주세요.
"""
        return prompt


# 사용 예시
if __name__ == "__main__":
    # 보고서 생성기 초기화
    generator = MOBYReportGenerator()
    
    # 예시 데이터 (실제로는 InfluxDB + ML 파이프라인에서 추출)
    report_data = {
        "metadata": {
            "period_start": "2025-11-07 09:00:00",
            "period_end": "2025-11-14 09:00:00",
            "equipment": "Conveyor A-01",
            "generated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        },
        
        "sensor_stats": {
            "temperature": {
                "mean": 38.2,
                "min": 22.1,
                "max": 52.3,
                "std": 6.8,
                "p95": 48.5,
                "missing_rate": 0.002,
                "threshold_violations": 12
            },
            "humidity": {
                "mean": 62.5,
                "min": 45.2,
                "max": 78.9,
                "std": 8.3,
                "p95": 75.1,
                "missing_rate": 0.002
            },
            "vibration": {
                "x": {"mean": 1.24, "peak": 3.87, "rms": 1.45, "p95": 2.31},
                "y": {"mean": 1.18, "peak": 3.52, "rms": 1.39, "p95": 2.18},
                "z": {"mean": 0.95, "peak": 2.89, "rms": 1.12, "p95": 1.87},
                "trend_note": "X축 진동이 주 후반부 약 15% 증가"
            },
            "accelerometer": {
                "x": {"mean": 0.12, "peak": 2.45, "rms": 0.38, "std": 0.34},
                "y": {"mean": -0.08, "peak": 2.21, "rms": 0.35, "std": 0.32},
                "z": {"mean": 9.81, "peak": 11.23, "rms": 9.85, "std": 0.28}
            },
            "gyroscope": {
                "x": {"mean": 0.03, "peak": 8.92, "rms": 1.24, "std": 1.22},
                "y": {"mean": -0.01, "peak": 7.68, "rms": 1.18, "std": 1.17},
                "z": {"mean": 0.02, "peak": 6.34, "rms": 0.95, "std": 0.94}
            },
            "sound": {
                "mean": 68.3,
                "max": 89.7,
                "p95": 81.5,
                "threshold_violations": 43
            },
            "pressure": {
                "mean": 1013.2,
                "min": 1008.5,
                "max": 1018.7,
                "trend_note": "주중 기압 완만 하강 (기상 영향)"
            }
        },
        
        "alarms": [
            {
                "timestamp": "2025-11-13 09:44:23",
                "sensor": "temperature",
                "value": 52.3,
                "threshold": 50.0,
                "level": "WARNING"
            },
            {
                "timestamp": "2025-11-12 18:22:11",
                "sensor": "sound",
                "value": 89.7,
                "threshold": 85.0,
                "level": "INFO"
            }
        ],
        
        "mlp_anomalies": [
            {
                "timestamp": "2025-11-11 14:32:18",
                "type": "Misalignment",
                "vector": [0.72, 0.18, 0.25],
                "vector_magnitude": 0.79,
                "threshold": 0.60,
                "component_labels": ["Misalignment", "Bearing", "Unbalance"]
            },
            {
                "timestamp": "2025-11-12 22:15:42",
                "type": "MLP_composite",
                "vector": [0.42, 0.38, 0.31],
                "vector_magnitude": 0.65,
                "threshold": 0.50,
                "component_labels": ["Misalignment", "Bearing", "Unbalance"]
            }
        ],
        
        "if_anomalies": [
            {
                "start_time": "2025-11-10 03:15:00",
                "end_time": "2025-11-10 03:42:00",
                "duration_minutes": 27,
                "anomaly_score": -0.18,
                "threshold": -0.15,
                "mlp_vector_magnitude": 0.22,
                "key_features": {
                    "vib_x_rms": {"normal_range": [1.2, 1.6], "measured": 2.34, "deviation_pct": 46},
                    "vib_y_peak": {"normal_range": [2.8, 3.8], "measured": 5.12, "deviation_pct": 35},
                    "gyro_z_std": {"normal_range": [0.8, 1.1], "measured": 1.87, "deviation_pct": 70}
                }
            }
        ],
        
        "correlations": {
            "temperature_vibration_x": {"value": 0.78, "interpretation": "강한 양의 상관"},
            "vibration_magnitude_sound": {"value": 0.65, "interpretation": "중간 양의 상관"},
            "temperature_humidity": {"value": -0.42, "interpretation": "중간 음의 상관"}
        }
    }
    
    # 보고서 생성
    print("보고서 생성 중...")
    report = generator.generate_report(report_data)
    
    # 파일로 저장
    output_filename = f"MOBY_Weekly_Report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(report)
    
    print(f"✓ 보고서 생성 완료: {output_filename}")
    print("\n" + "="*80)
    print(report[:500] + "...")  # 처음 500자 미리보기

보고서 생성 중...
✓ 보고서 생성 완료: MOBY_Weekly_Report_20251117_143412.md

# 📘 MOBY 설비 모니터링 · 이상 탐지 주간 보고서

| 항목 | 내용 |
| :--- | :--- |
| **보고 기간** | 2025-11-07 09:00:00 ~ 2025-11-14 09:00:00 |
| **설비** | Conveyor A-01 |
| **생성 일시** | 2025-11-17 14:33:44 |

---

## 1. 📊 Executive Summary

이번 주 Conveyor A-01 설비에서는 X축 진동이 주 후반부 약 15% 증가했으며, 이는 온도와 강한 양의 상관관계를 보여 마찰 또는 기계적 스트레스 증가를 시사합니다. 총 2건의 규칙 기반 알람(온도, 음압)과 3건의 이상 탐지(MLP 기반 2건, Isolation Forest 기반 1건)가 발생했습니다. 특히 Isolation Forest가 탐지한 미지의 이상은 진동 및 자이로스코프 데이터에서 심각한 편차를 보여 즉각적인 조사가 필요합니다. 전반적으로 설비의 기계적 상태에 대한 주의 깊은 모니터...
